In [2]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # グラフ作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=model_name)
    
    # tools を定義して llm.bind_tools() で紐付け
    tools = [TavilySearchResults()]
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノード
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # ノード追加
    graph_builder.add_node("chatbot", chatbot)
    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)

    # 条件付きエッジ
    graph_builder.add_conditional_edges("chatbot", tools_condition)

    # ツール→チャットボットへの戻りエッジ
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノード
    graph_builder.set_entry_point("chatbot")

    # 記憶保持用
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    
    return graph

# ===== グラフ実行関数 =====
# stream_graph_updates() に graph を渡す形に修正
def stream_graph_updates(graph: StateGraph, user_input: str):
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        # 最後のメッセージを表示
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
while True:
    user_input = input("質問：")
    if user_input.strip()=="":
        print("ありがとうございました！")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどんなことをお手伝いできますか？
１足す２は？
1足す2は3です。何か他に知りたいことはありますか？
台湾旅行について検索結果を教えて

[{"url": "https://www.sofhotel.com.tw/tw/blog/traveling-to-taiwan-heres-what-you-need-to-know.html", "content": "而且台灣人非常地友善。在大多數情況下，您只需要向當地人請求協助，都可以獲得親切的回應。當您在台灣旅遊時，也可能遇見熱情的當地人詢問您來自哪個國家，以及您是否喜歡他們的家鄉。\n\n當然，仍有少數可能會遇到標明不清楚的號誌、不會說英語的人，儘管如此，他們會盡力找到合適的人來協助您。\n\n開始規劃你的台灣之旅！\n\n旅客在規劃旅行時，特別需要注意的一點是：台灣比地圖上看起來大得多。許多初次來台的旅客認為台灣只有「台北和其他小地方」，或者認為可以在幾天內遊覽整個國家。事實上，要看遍台灣通常需要花費幾個星期的時間。\n\n如果想要探索島上全部的縣市，可以在國內停留至少兩個星期。若行程安排得很密集，也可以在一個星期內遊覽相當大的範圍，端看您如何安排行程。\n\n台灣是一個旅費相對便宜的國家。大多數旅客在這個國家旅行時，每人每天的預算大約在100美元左右。\n\n雖然台灣可能一開始不在你的計劃中，但它真的值得一遊。這座島嶼上的每一個角落，都能找到值得關注和探索的事物。台中SOF 植光花園酒店的團隊向您保證，您可以在一個融合了藝術、文化、建築和美食的大熔爐中度過人生中最美好的時光。 [...] 熱愛戶外活動和登山的人常會將台灣視為夢幻旅遊地，台灣特色地形所形成的陡峭山壁、飛瀉瀑布在一些國家難以見到。濃密的竹林讓遊客彷彿置身於童話故事中，而攀登高山峰頂時的廣闊視野更是無與倫比。\n\n廣闊的步道網絡縱橫交錯於整個島嶼，遊客可以更輕鬆地進入山林。雖然大多數的登山區域可以免費進入，但一些較具挑戰性的山岳仍需經過許可才能通過。\n\n預計前往山區遠足者應提早規劃，確保有足夠的時間並提交必要的文件、備妥用品，以免在行程期間浪費時間，造成不必要的失望。\n\n享受優質的公共Wi-Fi\n\n台灣政府建立了名為「iTaiwan」的 免費WiFi 服務，範圍幾乎涵蓋全台灣，使台灣在公共網路方面達到另一個